In [33]:
import re
import numpy as np
import pandas as pd
import requests
from bs4 import BeautifulSoup
import lxml

In [ ]:
html_page = requests.get('https://www.boxofficemojo.com/year/world/2019/', 'lxml')
soup = BeautifulSoup(html_page.content, 'lxml')

In [ ]:
movie_listings_container = soup.find('table', class_='mojo-body-table')
movies = movie_listings_container.find_all('tr')
movies.pop(0)

year_rank = movies[0].find_all('td')[0].text
title = movies[0].find_all('td')[1].text
worldwide = movies[0].find_all('td')[2].text
domestic = movies[0].find_all('td')[3].text
per_dom = movies[0].find_all('td')[4].text
foreign = movies[0].find_all('td')[5].text
per_for = movies[0].find_all('td')[6].text

moviepage_link = movies[0].find(class_='a-link-normal')['href']

moviepage_link_req = requests.get('https://www.boxofficemojo.com' + moviepage_link, 'lxml')
soup2 = BeautifulSoup(moviepage_link_req.content, 'lxml')

overview_tab_link = soup2.find('a', class_='mojo-title-link')['href']
overview_tab_link

overview_link_req = requests.get('https://www.boxofficemojo.com' + overview_tab_link, 'lxml')
soup3 = BeautifulSoup(overview_link_req.content, 'lxml')

budget_container = soup3.find('div', class_='mojo-summary-values')
budget = budget_container.find_all(class_='money')[1].text
budget_container

In [2]:
def scrape_movies(root_url, years):
    movies_list = []
    for year in years:
        html_page = requests.get(root_url + '{}/'.format(year), 'lxml')
        soup = BeautifulSoup(html_page.content, 'lxml')
        movie_listings_container = soup.find('table', class_='mojo-body-table')
        movies = movie_listings_container.find_all('tr')
        movies.pop(0) # First entry in the container on every page is the headers of the columns so I threw them out
        for movie in movies:
            try:
                keys = ['Year_Rank', 'Title', 'Worldwide_Gross', 'Domestic', 'Percent_Domestic',  
                        'Foreign', 'Percent_Foreign', 'Year']
                year_rank = movie.find_all('td')[0].text
                title = movie.find_all('td')[1].text
                worldwide = movie.find_all('td')[2].text
                domestic = movie.find_all('td')[3].text
                per_dom = movie.find_all('td')[4].text
                foreign = movie.find_all('td')[5].text
                per_for = movie.find_all('td')[6].text
                movie_year = year
                values = [year_rank, title, worldwide, domestic, per_dom, foreign, per_for, movie_year]
                movie_dict = dict(zip(keys,values))

                moviepage_link = movie.find(class_='a-link-normal')['href']
                moviepage_link_req = requests.get('https://www.boxofficemojo.com' + moviepage_link, 'lxml')
                soup2 = BeautifulSoup(moviepage_link_req.content, 'lxml')
                overview_tab_link = soup2.find('a', class_='mojo-title-link')['href']
                overview_link_req = requests.get('https://www.boxofficemojo.com' + overview_tab_link, 'lxml')
                soup3 = BeautifulSoup(overview_link_req.content, 'lxml')
                movieinfo_container = soup3.find('div', class_='mojo-summary-values')
                spans = movieinfo_container.find_all('span')
                spans_list = [span.text for span in spans]
                spans_list_final = []
                for span in spans_list:
                    if span not in spans_list_final:
                        spans_list_final.append(span)
                movieinfo_dict = dict(zip(spans_list_final[::2],spans_list_final[1::2]))
                entry_dict = {**movie_dict, **movieinfo_dict}
                movies_list.append(entry_dict)
            except:
                pass

    return movies_list

In [10]:
df = pd.DataFrame(scrape_movies('https://www.boxofficemojo.com/year/world/', range(2000,2005)))
df.to_csv(r'C:\Users\drudi\DataScience\Module01\Final Project\Module-1-Final-Project\BoxOfficMojoScrapeFinal2000-2004.csv')
df

,Year_Rank,Title,Worldwide_Gross,Domestic,Percent_Domestic,Foreign,Percent_Foreign,Year,Domestic Distributor,Domestic Opening,Budget,Earliest Release Date,MPAA,Running Time,Genres,\n IMDbPro\n
0,1,Mission: Impossible II,"$546,388,108","$215,409,889",39.4%,"$330,978,219",60.6%,2000,Paramount PicturesSee full company information...,"$57,845,297","$125,000,000","May 24, 2000\n (Domestic)",PG-13,2 hr 3 min,Action\n \n Adventure\n \n ...,See more details at IMDbPro\n\n
1,2,Gladiator,"$460,583,960","$187,705,427",40.8%,"$272,878,533",59.2%,2000,DreamWorks DistributionSee full company inform...,"$34,819,017","$103,000,000","May 4, 2000\n (Australia)",R,2 hr 35 min,Action\n \n Adventure\n \n ...,See more details at IMDbPro\n\n
2,3,Cast Away,"$429,632,142","$233,632,142",54.4%,"$196,000,000",45.6%,2000,Twentieth Century FoxSee full company informat...,"$28,883,406","$90,000,000","December 22, 2000\n (Domestic)",PG-13,2 hr 23 min,Adventure\n \n Drama\n \n ...,See more details at IMDbPro\n\n
3,4,What Women Want,"$374,111,707","$182,811,707",48.9%,"$191,300,000",51.1%,2000,Paramount PicturesSee full company information...,"$33,614,543","$70,000,000","December 15, 2000\n (Domestic)",PG-13,2 hr 7 min,Comedy\n \n Fantasy\n \n R...,See more details at IMDbPro\n\n
4,5,Dinosaur,"$349,822,765","$137,748,063",39.4%,"$212,074,702",60.6%,2000,Walt Disney Studios,"$38,854,851","$127,500,000","May 19, 2000\n (Domestic)",PG,1 hr 22 min,Adventure\n \n Animation\n \n ...,See more details at IMDbPro\n\n
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2350,551,Shwaas,"$1,416","$1,416",100%,-,-,2004,Kathi ArtsSee full company information\n\n,"$1,042",NaN,"December 10, 2004\n (Domestic)",NaN,1 hr 47 min,Drama,See more details at IMDbPro\n\n
2351,552,800 Bullets 2004 Re-release,$866,$866,100%,-,-,2004,NaN,NaN,NaN,"October 18, 2002\n (Spain)",R,2 hr 4 min,Action\n \n Comedy\n \n Dr...,See more details at IMDbPro\n\n
2352,553,Anatomy 2 2004 Re-release,$623,$623,100%,-,-,2004,NaN,NaN,NaN,"February 6, 2003\n (Germany)",R,1 hr 41 min,Horror\n \n Sci-Fi\n \n Th...,See more details at IMDbPro\n\n
2353,554,"Jesus, You Know",$604,$604,100%,-,-,2004,Leisure Time FeaturesSee full company informat...,$604,NaN,"December 3, 2004\n (Domestic)",NaN,1 hr 27 min,Documentary,See more details at IMDbPro\n\n


In [11]:
df2020 = pd.read_csv('BoxOfficMojoScrapeFinal2020.csv')
df2020

,Unnamed: 0,Year_Rank,Title,Worldwide_Gross,Domestic,Percent_Domestic,Foreign,Percent_Foreign,Year,Domestic Distributor,Domestic Opening,Budget,Earliest Release Date,MPAA,Running Time,Genres,\n IMDbPro\n
0,0,1,Bad Boys for Life,"$419,074,646","$204,417,855",48.8%,"$214,656,791",51.2%,2020,Sony Pictures Entertainment (SPE)See full comp...,"$62,504,105","$90,000,000","January 15, 2020\n (LATAM, APAC)",R,2 hr 4 min,Action\n \n Comedy\n \n Cr...,See more details at IMDbPro\n\n
1,1,2,Sonic the Hedgehog,"$306,766,470","$146,066,470",47.6%,"$160,700,000",52.4%,2020,Paramount PicturesSee full company information...,"$58,018,348","$85,000,000","February 12, 2020\n (APAC, EMEA)",PG,1 hr 39 min,Action\n \n Adventure\n \n ...,See more details at IMDbPro\n\n
2,2,3,Dolittle,"$224,752,486","$77,047,065",34.3%,"$147,705,421",65.7%,2020,Universal PicturesSee full company information...,"$21,844,045","$175,000,000","January 8, 2020\n (South Korea)",PG,1 hr 41 min,Adventure\n \n Comedy\n \n ...,See more details at IMDbPro\n\n
3,3,4,Birds of Prey: And the Fantabulous Emancipatio...,"$201,858,461","$84,158,461",41.7%,"$117,700,000",58.3%,2020,Warner Bros.See full company information\n\n,"$33,010,017","$84,500,000","February 5, 2020\n (APAC, EMEA)",R,1 hr 49 min,Action\n \n Adventure\n \n ...,See more details at IMDbPro\n\n
4,4,5,The Invisible Man,"$128,251,913","$64,914,050",50.6%,"$63,337,863",49.4%,2020,Universal PicturesSee full company information...,"$28,205,665","$7,000,000","February 26, 2020\n (EMEA, APAC)",R,2 hr 4 min,Horror\n \n Mystery\n \n S...,See more details at IMDbPro\n\n
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
155,155,158,The Fox Hunter,"$1,577","$1,577",100%,-,-,2020,Indican PicturesSee full company information\n\n,$602,NaN,"June 5, 2020\n (Domestic)",NaN,NaN,Comedy,See more details at IMDbPro\n\n
156,156,159,Hai Tang Hong,"$1,107","$1,107",100%,-,-,2020,M Star InternationalSee full company informati...,$310,NaN,"February 21, 2020\n (Domestic)",NaN,1 hr 30 min,NaN,See more details at IMDbPro\n\n
157,157,160,The Transcendents,"$1,090","$1,090",100%,-,-,2020,Indican PicturesSee full company information\n\n,$358,NaN,"June 26, 2020\n (Domestic)",NaN,1 hr 36 min,Music\n \n Mystery\n \n Th...,See more details at IMDbPro\n\n
158,158,161,Inception 10th Anniversary re-release,$387,-,-,$387,100%,2020,Warner Bros.See full company information\n\n,"$62,785,337","$160,000,000","July 15, 2010\n (EMEA, APAC)",PG-13,2 hr 28 min,Action\n \n Adventure\n \n ...,See more details at IMDbPro\n\n


In [13]:
df2019 = pd.read_csv('BoxOfficMojoScrapeFinal2019.csv')
df2019

,Unnamed: 0,Year_Rank,Title,Worldwide_Gross,Domestic,Percent_Domestic,Foreign,Percent_Foreign,Year,Domestic Distributor,Domestic Opening,Budget,Earliest Release Date,MPAA,Running Time,Genres,\n IMDbPro\n
0,0,1,Avengers: Endgame,"$2,797,800,564","$858,373,000",30.7%,"$1,939,427,564",69.3%,2019,Walt Disney Studios,"$357,115,007","$356,000,000","April 24, 2019\n (21 markets)",PG-13,3 hr 1 min,Action\n \n Adventure\n \n ...,See more details at IMDbPro\n\n
1,1,2,The Lion King,"$1,656,943,394","$543,638,043",32.8%,"$1,113,305,351",67.2%,2019,Walt Disney Studios,"$191,770,759","$260,000,000","July 11, 2019\n (Australia)",PG,1 hr 58 min,Adventure\n \n Animation\n \n ...,See more details at IMDbPro\n\n
2,2,3,Frozen II,"$1,450,026,933","$477,373,578",32.9%,"$972,653,355",67.1%,2019,Walt Disney Studios,"$130,263,358","$150,000,000","November 20, 2019\n (EMEA, APAC)",PG,1 hr 43 min,Adventure\n \n Animation\n \n ...,See more details at IMDbPro\n\n
3,3,4,Spider-Man: Far from Home,"$1,131,927,996","$390,532,085",34.5%,"$741,395,911",65.5%,2019,Sony Pictures Entertainment (SPE)See full comp...,"$92,579,212","$160,000,000","June 28, 2019\n (APAC, China)",PG-13,2 hr 9 min,Action\n \n Adventure\n \n ...,See more details at IMDbPro\n\n
4,4,5,Captain Marvel,"$1,128,274,794","$426,829,839",37.8%,"$701,444,955",62.2%,2019,Walt Disney Studios,"$153,433,423","$160,000,000","March 6, 2019\n (APAC, EMEA)",PG-13,2 hr 3 min,Action\n \n Adventure\n \n ...,See more details at IMDbPro\n\n
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
757,757,762,The Hours and Times,"$1,273","$1,273",100%,-,-,2019,OscilloscopeSee full company information\n\n,"$1,273",NaN,"March 1, 2019\n (Domestic)",NaN,57 min,Drama\n \n Music,See more details at IMDbPro\n\n
758,758,763,The Wolf Hour,"$1,255",-,-,"$1,255",100%,2019,Brainstorm MediaSee full company information\n\n,NaN,NaN,"December 6, 2019\n (Domestic)",R,1 hr 39 min,Drama\n \n Mystery\n \n Th...,See more details at IMDbPro\n\n
759,759,764,A Fish in the Bathtub 2019 re-release,"$1,237","$1,237",100%,-,-,2019,Cohen Media GroupSee full company information\n\n,"$1,237",NaN,"November 8, 2019\n (Domestic)",PG-13,1 hr 36 min,Comedy,See more details at IMDbPro\n\n
760,760,765,The Untold Story,$790,$790,100%,-,-,2019,Ammo ContentSee full company information\n\n,$790,NaN,"January 18, 2019\n (Domestic)",NaN,1 hr 44 min,Comedy\n \n Drama,See more details at IMDbPro\n\n


In [14]:
df2018 = pd.read_csv('BoxOfficMojoScrapeFinal2018.csv')
df2018

,Unnamed: 0,Year_Rank,Title,Worldwide_Gross,Domestic,Percent_Domestic,Foreign,Percent_Foreign,Year,Domestic Distributor,Domestic Opening,Earliest Release Date,MPAA,Running Time,Genres,\n IMDbPro\n,Budget
0,0,1,Avengers: Infinity War,"$2,048,359,754","$678,815,482",33.1%,"$1,369,544,272",66.9%,2018,Walt Disney Studios,"$257,698,183","April 25, 2018\n (APAC, EMEA)",PG-13,2 hr 29 min,Action\n \n Adventure\n \n ...,See more details at IMDbPro\n\n,NaN
1,1,2,Black Panther,"$1,346,913,161","$700,059,566",52%,"$646,853,595",48%,2018,Walt Disney Studios,"$202,003,951","February 13, 2018\n (EMEA, APAC)",PG-13,2 hr 14 min,Action\n \n Adventure\n \n ...,See more details at IMDbPro\n\n,NaN
2,2,3,Jurassic World: Fallen Kingdom,"$1,308,467,944","$417,719,760",31.9%,"$890,748,184",68.1%,2018,Universal PicturesSee full company information...,"$148,024,610","June 6, 2018\n (EMEA, APAC)",PG-13,2 hr 8 min,Action\n \n Adventure\n \n ...,See more details at IMDbPro\n\n,"$170,000,000"
3,3,4,Incredibles 2,"$1,242,805,359","$608,581,744",49%,"$634,223,615",51%,2018,Walt Disney Studios,"$182,687,905","June 14, 2018\n (16 markets)",PG,1 hr 58 min,Action\n \n Adventure\n \n ...,See more details at IMDbPro\n\n,NaN
4,4,5,Aquaman,"$1,148,461,807","$335,061,807",29.2%,"$813,400,000",70.8%,2018,Warner Bros.See full company information\n\n,"$67,873,522","December 7, 2018\n (China)",PG-13,2 hr 23 min,Action\n \n Adventure\n \n ...,See more details at IMDbPro\n\n,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
876,876,884,Under the Turban,"$1,190","$1,190",100%,-,-,2018,GathrFilmsSee full company information\n\n,NaN,"April 25, 2018\n (Domestic)",NaN,1 hr 29 min,Documentary,See more details at IMDbPro\n\n,NaN
877,877,885,Half Brothers,"$1,100","$1,100",100%,-,-,2018,Parade Deck FilmsSee full company information\n\n,"$1,100","September 21, 2018\n (Domestic)",NaN,1 hr 37 min,Comedy\n \n Drama,See more details at IMDbPro\n\n,NaN
878,878,886,Invisible Hands,"$1,034","$1,034",100%,-,-,2018,First RunSee full company information\n\n,$456,"November 23, 2018\n (Domestic)",NaN,1 hr 20 min,Documentary,See more details at IMDbPro\n\n,NaN
879,879,887,TVTV: Video Revolutionaries,$812,$812,100%,-,-,2018,First RunSee full company information\n\n,$556,"October 19, 2018\n (Domestic)",NaN,1 hr 22 min,Documentary,See more details at IMDbPro\n\n,NaN


In [15]:
df2017 = pd.read_csv('BoxOfficMojoScrapeFinal2017.csv')
df2017

,Unnamed: 0,Year_Rank,Title,Worldwide_Gross,Domestic,Percent_Domestic,Foreign,Percent_Foreign,Year,Domestic Distributor,Domestic Opening,Budget,Earliest Release Date,MPAA,Running Time,Genres,\n IMDbPro\n
0,0,1,Star Wars: Episode VIII - The Last Jedi,"$1,332,539,889","$620,181,382",46.5%,"$712,358,507",53.5%,2017,Walt Disney Studios,"$220,009,584","$317,000,000","December 13, 2017\n (APAC, EMEA)",PG-13,2 hr 32 min,Action\n \n Adventure\n \n ...,See more details at IMDbPro\n\n
1,1,2,Beauty and the Beast,"$1,263,521,126","$504,014,165",39.9%,"$759,506,961",60.1%,2017,Walt Disney Studios,"$174,750,616","$160,000,000","March 16, 2017\n (24 markets)",PG,2 hr 9 min,Family\n \n Fantasy\n \n M...,See more details at IMDbPro\n\n
2,2,3,The Fate of the Furious,"$1,236,005,118","$226,008,385",18.3%,"$1,009,996,733",81.7%,2017,Universal PicturesSee full company information...,"$98,786,705","$250,000,000","April 12, 2017\n (9 markets)",PG-13,2 hr 16 min,Action\n \n Adventure\n \n ...,See more details at IMDbPro\n\n
3,3,4,Despicable Me 3,"$1,034,799,409","$264,624,300",25.6%,"$770,175,109",74.4%,2017,Universal PicturesSee full company information...,"$72,434,025","$80,000,000","June 14, 2017\n (Philippines)",PG,1 hr 29 min,Adventure\n \n Animation\n \n ...,See more details at IMDbPro\n\n
4,4,5,Jumanji: Welcome to the Jungle,"$962,077,546","$404,515,480",42%,"$557,562,066",58%,2017,Sony Pictures Entertainment (SPE)See full comp...,"$36,169,328","$90,000,000","December 20, 2017\n (8 markets)",PG-13,1 hr 59 min,Action\n \n Adventure\n \n ...,See more details at IMDbPro\n\n
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
774,774,777,A Different American Dream,"$1,172","$1,172",100%,-,-,2017,First RunSee full company information\n\n,$848,NaN,"January 6, 2017\n (Domestic)",NaN,1 hr 24 min,Documentary,See more details at IMDbPro\n\n
775,775,778,The Penguin Counters,$976,$976,100%,-,-,2017,First RunSee full company information\n\n,$976,NaN,"April 21, 2017\n (Domestic)",NaN,1 hr 7 min,Adventure\n \n Documentary,See more details at IMDbPro\n\n
776,776,779,Extraordinary Ordinary People,$880,$880,100%,-,-,2017,First RunSee full company information\n\n,$744,NaN,"September 15, 2017\n (Domestic)",NaN,1 hr 24 min,Documentary,See more details at IMDbPro\n\n
777,777,780,Antiporno,$294,-,-,$294,100%,2017,MUBISee full company information\n\n,NaN,NaN,"November 24, 2017\n (Domestic)",NaN,1 hr 16 min,Drama,See more details at IMDbPro\n\n


In [16]:
df2016 = pd.read_csv('BoxOfficMojoScrapeFinal2016.csv')
df2016

,Unnamed: 0,Year_Rank,Title,Worldwide_Gross,Domestic,Percent_Domestic,Foreign,Percent_Foreign,Year,Domestic Distributor,Domestic Opening,Budget,Earliest Release Date,MPAA,Running Time,Genres,\n IMDbPro\n
0,0,1,Captain America: Civil War,"$1,153,296,293","$408,084,349",35.4%,"$745,211,944",64.6%,2016,Walt Disney Studios,"$179,139,142","$250,000,000","April 27, 2016\n (APAC, EMEA)",PG-13,2 hr 27 min,Action\n \n Adventure\n \n ...,See more details at IMDbPro\n\n
1,1,2,Rogue One: A Star Wars Story,"$1,056,057,273","$532,177,324",50.4%,"$523,879,949",49.6%,2016,Walt Disney Studios,"$155,081,681","$200,000,000","December 14, 2016\n (EMEA)",PG-13,2 hr 13 min,Action\n \n Adventure\n \n ...,See more details at IMDbPro\n\n
2,2,3,Finding Dory,"$1,028,570,889","$486,295,561",47.3%,"$542,275,328",52.7%,2016,Walt Disney Studios,"$135,060,273",NaN,"June 15, 2016\n (Philippines)",PG,1 hr 37 min,Adventure\n \n Animation\n \n ...,See more details at IMDbPro\n\n
3,3,4,Zootopia,"$1,023,784,195","$341,268,248",33.3%,"$682,515,947",66.7%,2016,Walt Disney Studios,"$75,063,401",NaN,"February 10, 2016\n (Belgium)",PG,1 hr 48 min,Adventure\n \n Animation\n \n ...,See more details at IMDbPro\n\n
4,4,5,The Jungle Book,"$966,550,600","$364,001,123",37.7%,"$602,549,477",62.3%,2016,Walt Disney Studios,"$103,261,464","$175,000,000","April 7, 2016\n (APAC, LATAM)",PG,1 hr 46 min,Adventure\n \n Drama\n \n ...,See more details at IMDbPro\n\n
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
758,758,759,Above and Below,"$1,189",-,-,"$1,189",100%,2016,OscilloscopeSee full company information\n\n,NaN,NaN,"April 15, 2016\n (Domestic, EMEA)",NaN,1 hr 58 min,Adventure\n \n Documentary\n \n ...,See more details at IMDbPro\n\n
759,759,760,Officer Downe,$850,$850,100%,-,-,2016,Magnolia PicturesSee full company information\n\n,$470,NaN,"November 18, 2016\n (Domestic)",R,1 hr 28 min,Action\n \n Comedy\n \n Sc...,See more details at IMDbPro\n\n
760,760,761,A Girl Like Grace,$713,$713,100%,-,-,2016,GVN ReleasingSee full company information\n\n,$458,NaN,"December 2, 2016\n (Domestic)",NaN,1 hr 33 min,Drama,See more details at IMDbPro\n\n
761,761,762,The Pit,$560,$560,100%,-,-,2016,Kino LorberSee full company information\n\n,NaN,NaN,"September 30, 2016\n (Domestic)",NaN,1 hr 36 min,Horror\n \n Mystery,See more details at IMDbPro\n\n


In [17]:
df2010_2015 = pd.read_csv('BoxOfficMojoScrapeFinal2010-2015.csv')
df2010_2015

,Unnamed: 0,Year_Rank,Title,Worldwide_Gross,Domestic,Percent_Domestic,Foreign,Percent_Foreign,Year,Domestic Distributor,Domestic Opening,Budget,Earliest Release Date,Running Time,Genres,\n IMDbPro\n,MPAA
0,0,1,Toy Story 3,"$1,066,969,703","$415,004,880",38.9%,"$651,964,823",61.1%,2010,Walt Disney Studios,"$110,307,189","$200,000,000","June 16, 2010\n (Egypt)",1 hr 43 min,Adventure\n \n Animation\n \n ...,See more details at IMDbPro\n\n,NaN
1,1,2,Alice in Wonderland,"$1,025,467,110","$334,191,110",32.6%,"$691,276,000",67.4%,2010,Walt Disney Studios,"$116,101,023","$200,000,000","March 3, 2010\n (EMEA)",1 hr 48 min,Adventure\n \n Family\n \n ...,See more details at IMDbPro\n\n,PG
2,2,3,Harry Potter and the Deathly Hallows: Part 1,"$976,536,918","$295,983,305",30.3%,"$680,553,613",69.7%,2010,Warner Bros.See full company information\n\n,"$125,017,372",NaN,"November 17, 2010\n (EMEA, APAC)",2 hr 26 min,Adventure\n \n Family\n \n ...,See more details at IMDbPro\n\n,PG-13
3,3,4,Inception,"$828,322,032","$292,576,195",35.3%,"$535,745,837",64.7%,2010,Warner Bros.See full company information\n\n,"$62,785,337","$160,000,000","July 15, 2010\n (APAC, EMEA)",2 hr 28 min,Action\n \n Adventure\n \n ...,See more details at IMDbPro\n\n,PG-13
4,4,5,Shrek Forever After,"$752,600,867","$238,736,787",31.7%,"$513,864,080",68.3%,2010,DreamWorksSee full company information\n\n,"$70,838,207","$165,000,000","May 20, 2010\n (EMEA, APAC)",1 hr 33 min,Adventure\n \n Animation\n \n ...,See more details at IMDbPro\n\n,PG
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4028,4028,729,It's All So Quiet 2015 Re-release,$724,$724,100%,-,-,2015,NaN,NaN,NaN,"April 25, 2013\n (Netherlands)",1 hr 33 min,Drama,See more details at IMDbPro\n\n,NaN
4029,4029,730,It's Hard Being Loved by Jerks 2015 Re-release,$697,$697,100%,-,-,2015,NaN,NaN,NaN,"September 17, 2008\n (EMEA)",1 hr 50 min,Documentary,See more details at IMDbPro\n\n,NaN
4030,4030,731,The 11th Hour,$510,-,-,$510,100%,2015,Brainstorm MediaSee full company information\n\n,NaN,NaN,"June 12, 2015\n (Domestic)",1 hr 37 min,Drama\n \n Thriller,See more details at IMDbPro\n\n,NaN
4031,4031,732,Paranoid Girls,$78,$78,100%,-,-,2015,NaN,$54,NaN,"November 13, 2015\n (Domestic)",1 hr 40 min,Comedy,See more details at IMDbPro\n\n,NaN


In [18]:
df2005_2009 = pd.read_csv('BoxOfficMojoScrapeFinal2005-2009.csv')
df2005_2009

,Unnamed: 0,Year_Rank,Title,Worldwide_Gross,Domestic,Percent_Domestic,Foreign,Percent_Foreign,Year,Domestic Distributor,Domestic Opening,Budget,Earliest Release Date,MPAA,Running Time,Genres,\n IMDbPro\n
0,0,1,Harry Potter and the Goblet of Fire,"$895,921,036","$290,013,036",32.4%,"$605,908,000",67.6%,2005,Warner Bros.See full company information\n\n,"$102,685,961","$150,000,000","November 16, 2005\n (United Arab Em...",PG-13,2 hr 37 min,Adventure\n \n Family\n \n ...,See more details at IMDbPro\n\n
1,1,2,Star Wars: Episode III - Revenge of the Sith,"$868,352,530","$380,270,577",43.8%,"$488,081,953",56.2%,2005,Twentieth Century FoxSee full company informat...,"$108,435,841","$113,000,000","May 18, 2005\n (EMEA)",PG-13,2 hr 20 min,Action\n \n Adventure\n \n ...,See more details at IMDbPro\n\n
2,2,3,"The Chronicles of Narnia: The Lion, the Witch ...","$745,013,115","$291,710,957",39.2%,"$453,302,158",60.8%,2005,Walt Disney Studios,"$65,556,312","$180,000,000","December 7, 2005\n (EMEA)",PG,2 hr 23 min,Adventure\n \n Family\n \n ...,See more details at IMDbPro\n\n
3,3,4,War of the Worlds,"$603,873,119","$234,280,354",38.8%,"$369,592,765",61.2%,2005,Paramount PicturesSee full company information...,"$64,878,725","$132,000,000","June 29, 2005\n (46 markets)",PG-13,1 hr 56 min,Adventure\n \n Sci-Fi\n \n ...,See more details at IMDbPro\n\n
4,4,5,King Kong,"$562,363,449","$218,080,025",38.8%,"$344,283,424",61.2%,2005,Universal PicturesSee full company information...,"$50,130,145","$207,000,000","December 14, 2005\n (38 markets)",PG-13,3 hr 7 min,Action\n \n Adventure\n \n ...,See more details at IMDbPro\n\n
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2990,2990,516,Confessionsofa Ex-Doofus-ItchyFooted Mutha,"$1,013","$1,013",100%,-,-,2009,Yeah,$604,NaN,"August 21, 2009\n (Domestic)",NaN,1 hr 39 min,Adventure\n \n Comedy,See more details at IMDbPro\n\n
2991,2991,517,How to Be a Serial Killer,$899,$899,100%,-,-,2009,Monterey MediaSee full company information\n\n,$899,NaN,"July 24, 2009\n (Domestic)",R,1 hr 31 min,Comedy\n \n Crime\n \n Horror,See more details at IMDbPro\n\n
2992,2992,518,All That Heaven Allows,$170,-,-,$170,100%,2009,Universal PicturesSee full company information...,NaN,NaN,"July 1, 2009\n (France)",NaN,1 hr 29 min,Drama\n \n Romance,See more details at IMDbPro\n\n
2993,2993,519,The Objective,$95,$95,100%,-,-,2009,IFC FilmsSee full company information\n\n,$95,NaN,"February 4, 2009\n (Domestic)",NaN,1 hr 30 min,Horror\n \n Sci-Fi\n \n Th...,See more details at IMDbPro\n\n


In [19]:
df2000_2004 = pd.read_csv('BoxOfficMojoScrapeFinal2000-2004.csv')
df2000_2004

,Unnamed: 0,Year_Rank,Title,Worldwide_Gross,Domestic,Percent_Domestic,Foreign,Percent_Foreign,Year,Domestic Distributor,Domestic Opening,Budget,Earliest Release Date,MPAA,Running Time,Genres,\n IMDbPro\n
0,0,1,Mission: Impossible II,"$546,388,108","$215,409,889",39.4%,"$330,978,219",60.6%,2000,Paramount PicturesSee full company information...,"$57,845,297","$125,000,000","May 24, 2000\n (Domestic)",PG-13,2 hr 3 min,Action\n \n Adventure\n \n ...,See more details at IMDbPro\n\n
1,1,2,Gladiator,"$460,583,960","$187,705,427",40.8%,"$272,878,533",59.2%,2000,DreamWorks DistributionSee full company inform...,"$34,819,017","$103,000,000","May 4, 2000\n (Australia)",R,2 hr 35 min,Action\n \n Adventure\n \n ...,See more details at IMDbPro\n\n
2,2,3,Cast Away,"$429,632,142","$233,632,142",54.4%,"$196,000,000",45.6%,2000,Twentieth Century FoxSee full company informat...,"$28,883,406","$90,000,000","December 22, 2000\n (Domestic)",PG-13,2 hr 23 min,Adventure\n \n Drama\n \n ...,See more details at IMDbPro\n\n
3,3,4,What Women Want,"$374,111,707","$182,811,707",48.9%,"$191,300,000",51.1%,2000,Paramount PicturesSee full company information...,"$33,614,543","$70,000,000","December 15, 2000\n (Domestic)",PG-13,2 hr 7 min,Comedy\n \n Fantasy\n \n R...,See more details at IMDbPro\n\n
4,4,5,Dinosaur,"$349,822,765","$137,748,063",39.4%,"$212,074,702",60.6%,2000,Walt Disney Studios,"$38,854,851","$127,500,000","May 19, 2000\n (Domestic)",PG,1 hr 22 min,Adventure\n \n Animation\n \n ...,See more details at IMDbPro\n\n
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2350,2350,551,Shwaas,"$1,416","$1,416",100%,-,-,2004,Kathi ArtsSee full company information\n\n,"$1,042",NaN,"December 10, 2004\n (Domestic)",NaN,1 hr 47 min,Drama,See more details at IMDbPro\n\n
2351,2351,552,800 Bullets 2004 Re-release,$866,$866,100%,-,-,2004,NaN,NaN,NaN,"October 18, 2002\n (Spain)",R,2 hr 4 min,Action\n \n Comedy\n \n Dr...,See more details at IMDbPro\n\n
2352,2352,553,Anatomy 2 2004 Re-release,$623,$623,100%,-,-,2004,NaN,NaN,NaN,"February 6, 2003\n (Germany)",R,1 hr 41 min,Horror\n \n Sci-Fi\n \n Th...,See more details at IMDbPro\n\n
2353,2353,554,"Jesus, You Know",$604,$604,100%,-,-,2004,Leisure Time FeaturesSee full company informat...,$604,NaN,"December 3, 2004\n (Domestic)",NaN,1 hr 27 min,Documentary,See more details at IMDbPro\n\n


In [22]:
frames = [df2020, df2019, df2018, df2017, df2016, df2010_2015, df2005_2009, df2000_2004]
df = pd.concat(frames)
df

,Unnamed: 0,Year_Rank,Title,Worldwide_Gross,Domestic,Percent_Domestic,Foreign,Percent_Foreign,Year,Domestic Distributor,Domestic Opening,Budget,Earliest Release Date,MPAA,Running Time,Genres,\n IMDbPro\n
0,0,1,Bad Boys for Life,"$419,074,646","$204,417,855",48.8%,"$214,656,791",51.2%,2020,Sony Pictures Entertainment (SPE)See full comp...,"$62,504,105","$90,000,000","January 15, 2020\n (LATAM, APAC)",R,2 hr 4 min,Action\n \n Comedy\n \n Cr...,See more details at IMDbPro\n\n
1,1,2,Sonic the Hedgehog,"$306,766,470","$146,066,470",47.6%,"$160,700,000",52.4%,2020,Paramount PicturesSee full company information...,"$58,018,348","$85,000,000","February 12, 2020\n (APAC, EMEA)",PG,1 hr 39 min,Action\n \n Adventure\n \n ...,See more details at IMDbPro\n\n
2,2,3,Dolittle,"$224,752,486","$77,047,065",34.3%,"$147,705,421",65.7%,2020,Universal PicturesSee full company information...,"$21,844,045","$175,000,000","January 8, 2020\n (South Korea)",PG,1 hr 41 min,Adventure\n \n Comedy\n \n ...,See more details at IMDbPro\n\n
3,3,4,Birds of Prey: And the Fantabulous Emancipatio...,"$201,858,461","$84,158,461",41.7%,"$117,700,000",58.3%,2020,Warner Bros.See full company information\n\n,"$33,010,017","$84,500,000","February 5, 2020\n (APAC, EMEA)",R,1 hr 49 min,Action\n \n Adventure\n \n ...,See more details at IMDbPro\n\n
4,4,5,The Invisible Man,"$128,251,913","$64,914,050",50.6%,"$63,337,863",49.4%,2020,Universal PicturesSee full company information...,"$28,205,665","$7,000,000","February 26, 2020\n (EMEA, APAC)",R,2 hr 4 min,Horror\n \n Mystery\n \n S...,See more details at IMDbPro\n\n
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2350,2350,551,Shwaas,"$1,416","$1,416",100%,-,-,2004,Kathi ArtsSee full company information\n\n,"$1,042",NaN,"December 10, 2004\n (Domestic)",NaN,1 hr 47 min,Drama,See more details at IMDbPro\n\n
2351,2351,552,800 Bullets 2004 Re-release,$866,$866,100%,-,-,2004,NaN,NaN,NaN,"October 18, 2002\n (Spain)",R,2 hr 4 min,Action\n \n Comedy\n \n Dr...,See more details at IMDbPro\n\n
2352,2352,553,Anatomy 2 2004 Re-release,$623,$623,100%,-,-,2004,NaN,NaN,NaN,"February 6, 2003\n (Germany)",R,1 hr 41 min,Horror\n \n Sci-Fi\n \n Th...,See more details at IMDbPro\n\n
2353,2353,554,"Jesus, You Know",$604,$604,100%,-,-,2004,Leisure Time FeaturesSee full company informat...,$604,NaN,"December 3, 2004\n (Domestic)",NaN,1 hr 27 min,Documentary,See more details at IMDbPro\n\n


In [23]:
df.to_csv(r'C:\Users\drudi\DataScience\Module01\Final Project\Module-1-Final-Project\BoxOfficeMojoScrapeFinal.csv')